In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from photostim_deve.control_exp.io import get_med_img_s2p, get_seg_img_cp, write_mp_file, write_gpl_file
from photostim_deve.control_exp.plot   import plot_fov, plot_fov_meds

%load_ext autoreload
%autoreload 2

In [ ]:
# Parameters
data_path = 'data_proc/jm/jm059/2025-10-28_a/'

use_seg = 's2p' # 's2p' or 'cp' (TODO: implement cp with the get_seg_img_cp() function)

n_cells_choose = 30 # number of cells to choose for stimulation
seed = 123

# visualisation parameters
vmax = 800 # saturation

In [ ]:
# The parameters underneath are used to generate the MarkPoints and Galvo Point List files.
# They should usually not be changed for a fixed imaging configuration (e. g. 512x512 FOV, 1.5 zoom, 30Hz frame rate, 15 um spiral size, 50 ms duration).

fov_shape = (512, 512)

# paths to templates
mp_temp_path = 'utils/templates/MarkPoints_template.xml'
gpl_temp_path = 'utils/templates/galvo_point_list_template.gpl'

# MarkPoints parameters (.xml file)
SpiralWidth = '0.0199325637636341'     # as proportion of the FOV
SpiralHeight = '0.0199325637636341'    # as proportion of the FOV
SpiralSizeInMicrons = '15.0000000000001'        # in microns

# Galvo point list parameters (.gpl file) - These should usually not be changed!
ActivityType = "MarkPoints" 
UncagingLaser = "Uncaging" 
UncagingLaserPower = "1000" 
Duration = "50" 
IsSpiral = "True" 
SpiralSize = "0.110870362837845" 
SpiralRevolutions = "7" 
Z = "807.424999999999"
X_lim = 2.79639654844993 # determined empirically by putting points in the corners of the FOV (I think this is voltage control to the galvo)
Y_lim = 3.09924006097119 # determined empirically by putting points in the corners of the FOV (I think this is voltage control to the galvo)

In [ ]:
mouse_str = data_path.split('/')[2]
export_path = os.path.join('export', mouse_str)
if not os.path.exists(export_path):
    os.makedirs(export_path)

In [ ]:
meds, mn_image, s2p_idxs, ops, f = get_med_img_s2p(data_path)


In [ ]:
# now randomly choose 30 cells
np.random.seed(seed)
inds = np.random.choice(len(meds), n_cells_choose, replace=False)
print(inds)

In [ ]:
plot_fov(mn_image, export_path, vmax=vmax, use_seg=use_seg)
plot_fov_meds(mn_image, meds, inds, export_path, vmax=vmax, use_seg=use_seg)

In [ ]:
# print the coordinates of the medians of the cells
print('Medians of the ROIs (in randomised order):')
# randomise the order of the indices
inds = np.random.permutation(inds)
count = 1 # bruker uses 1-based indexing
for i in inds:
    # add leading zeros to the index
    i_str = str(i).zfill(3)
    print(f'point: {count} ROI {i_str}: {meds[i]}')
    count += 1

In [ ]:
write_mp_file(meds,
              inds,
              mp_temp_path=mp_temp_path, 
              mouse_str=mouse_str,
              export_path=export_path, 
              fov_shape=fov_shape, 
              SpiralWidth=SpiralWidth, 
              SpiralHeight=SpiralHeight, 
              SpiralSizeInMicrons=SpiralSizeInMicrons,
              use_seg=use_seg)

In [ ]:
write_gpl_file(meds, 
                inds,
                gpl_temp_path=gpl_temp_path, 
                mouse_str=mouse_str,
                export_path=export_path, 
                fov_shape=fov_shape, 
                ActivityType=ActivityType, 
                UncagingLaser=UncagingLaser, 
                UncagingLaserPower=UncagingLaserPower, 
                Duration=Duration, 
                IsSpiral=IsSpiral, 
                SpiralSize=SpiralSize, 
                SpiralRevolutions=SpiralRevolutions, 
                Z=Z, 
                X_lim=X_lim, 
                Y_lim=Y_lim,
                use_seg=use_seg)